In [171]:
import numpy as np
import pandas as pd
from pathlib import Path
import json

# need to cite things!

In [172]:
def transcript_truncation(data_frame):
    # TODO: make type hints/docstrings for this function
    ending_char = ['.','!','?','"']
    for i in data_frame.index:
        string = data_frame.loc[i, 'Text']
        index = max([string.rfind(char) for char in ending_char])
        if index == -1:
            index = len(string)
        data_frame.loc[i, 'Text'] = string[0:index+1]

    return data_frame

In [173]:
hanna_transcripts = pd.read_csv('raw_data/hanna_stories_annotations.csv')
hanna_transcripts = hanna_transcripts.drop(['Name', 
                                           'Worker ID', 
                                           'Assignment ID', 
                                           'Work time in seconds',
                                           'Relevance',
                                           'Empathy',
                                           'Surprise',
                                           'Engagement',
                                           'Complexity',
                                        ],
                                        axis = 1
)
display(hanna_transcripts)

,Story ID,Prompt,Human,Story,Model,Coherence
0,0,When you die the afterlife is an arena where y...,"3,000 years have I been fighting. Every mornin...","3,000 years have I been fighting. Every mornin...",Human,4
1,0,When you die the afterlife is an arena where y...,"3,000 years have I been fighting. Every mornin...","3,000 years have I been fighting. Every mornin...",Human,5
2,0,When you die the afterlife is an arena where y...,"3,000 years have I been fighting. Every mornin...","3,000 years have I been fighting. Every mornin...",Human,2
3,1,A new law is enacted that erases soldiers memo...,"“Dad, you 're on TV again !” I heard Eric 's v...","“Dad, you 're on TV again !” I heard Eric 's v...",Human,5
4,1,A new law is enacted that erases soldiers memo...,"“Dad, you 're on TV again !” I heard Eric 's v...","“Dad, you 're on TV again !” I heard Eric 's v...",Human,4
...,...,...,...,...,...,...
3163,1054,"When a new president is elected, they are give...",“Mr President I want you to know I am telling ...,'said a puppet'President Bush stopped the old ...,TD-VAE,1
3164,1054,"When a new president is elected, they are give...",“Mr President I want you to know I am telling ...,'said a puppet'President Bush stopped the old ...,TD-VAE,2
3165,1055,You discover a grand hall filled with legendar...,"Waking with a start, my blankets strewn wildly...",It is YOU. Your mother's greatest love.”...'Oh...,TD-VAE,4
3166,1055,You discover a grand hall filled with legendar...,"Waking with a start, my blankets strewn wildly...",It is YOU. Your mother's greatest love.”...'Oh...,TD-VAE,5


In [174]:
hanna_transcripts_cleaned = pd.DataFrame(columns=hanna_transcripts.columns)
for x in hanna_transcripts['Story ID'].unique():
    hanna_transcripts_x = hanna_transcripts[hanna_transcripts['Story ID'] == x]
    hanna_transcripts_cleaned.loc[len(hanna_transcripts_cleaned)] = hanna_transcripts_x.iloc[0]
    hanna_transcripts_cleaned.loc[x, 'Coherence'] = np.floor(hanna_transcripts_x['Coherence'].mean())
display(hanna_transcripts_cleaned)

,Story ID,Prompt,Human,Story,Model,Coherence
0,0,When you die the afterlife is an arena where y...,"3,000 years have I been fighting. Every mornin...","3,000 years have I been fighting. Every mornin...",Human,3.0
1,1,A new law is enacted that erases soldiers memo...,"“Dad, you 're on TV again !” I heard Eric 's v...","“Dad, you 're on TV again !” I heard Eric 's v...",Human,4.0
2,2,A scientific study proves that all humans have...,"When Tyler entered the ward, his daughter Vale...","When Tyler entered the ward, his daughter Vale...",Human,4.0
3,3,Write a story about an elderly wizard and his ...,His body was failing. He had taken care of it ...,His body was failing. He had taken care of it ...,Human,4.0
4,4,"You have become death, destroyer of worlds.","I saw the button. It was simple, red, no words...","I saw the button. It was simple, red, no words...",Human,5.0
...,...,...,...,...,...,...
1051,1051,All you hear is a dripping noise.,All I hear is a dripping noise. At first it wa...,"' I want no more.' she cried, tossing her cloa...",TD-VAE,2.0
1052,1052,Google search engine always was alive. They re...,86388: *what am i* 86389: They ask me question...,'it says. 'it repeats in every language the wo...,TD-VAE,2.0
1053,1053,Heart of Eden,"If everything went perfectly right, her creati...",opens almost a month after the start of Star T...,TD-VAE,3.0
1054,1054,"When a new president is elected, they are give...",“Mr President I want you to know I am telling ...,'said a puppet'President Bush stopped the old ...,TD-VAE,2.0


In [175]:
hanna_transcripts_restructured = hanna_transcripts_cleaned.drop(['Story ID', 'Prompt', 'Human', 'Model'], axis=1).rename(columns={'Story':'Text'})
hanna_transcripts_restructured = transcript_truncation(hanna_transcripts_restructured)
display(hanna_transcripts_restructured)

,Text,Coherence
0,"3,000 years have I been fighting. Every mornin...",3.0
1,"“Dad, you 're on TV again !” I heard Eric 's v...",4.0
2,"When Tyler entered the ward, his daughter Vale...",4.0
3,His body was failing. He had taken care of it ...,4.0
4,"I saw the button. It was simple, red, no words...",5.0
...,...,...
1051,"' I want no more.' she cried, tossing her cloa...",2.0
1052,'it says. 'it repeats in every language the wo...,2.0
1053,opens almost a month after the start of Star T...,3.0
1054,'said a puppet'President Bush stopped the old ...,2.0


In [176]:
with open('raw_data/cohesentia_stories_1.json', 'r') as file:
        cohesentia_stories_1 = json.load(file)
with open('raw_data/cohesentia_stories_2.json', 'r') as file:
        cohesentia_stories_2 = json.load(file)
cohesentia_transcripts = pd.concat([pd.DataFrame(cohesentia_stories_1).T,pd.DataFrame(cohesentia_stories_2).T]).sort_values(by = 'StoryID')
cohesentia_transcripts['Coherence'] = [cohesentia_transcripts['HolisticData'].iloc[i[0]]['consensus_score'] for i in enumerate(cohesentia_transcripts['StoryID'])]
display(cohesentia_transcripts)

,StoryID,Title,Text,IncrementalData,HolisticData,Coherence
0,0,David Drops the Weight,David always knew he had a lot of weight to lo...,{'sentences': ['David always knew he had a lot...,"{'num_annotators': 2, 'scores': {'annot0': 2.0...",2
1,1,Lost in a World of Adventure,I was curious about the world and I wanted to ...,{'sentences': ['I was curious about the world ...,"{'num_annotators': 2, 'scores': {'annot0': 2.0...",2
2,2,Deadly,"Once upon a time, there were seven deadly sins...","{'sentences': ['Once upon a time, there were s...","{'num_annotators': 2, 'scores': {'annot0': 3.0...",2
3,3,A Story of Magic,Sally was a very special little girl. She had ...,{'sentences': ['Sally was a very special littl...,"{'num_annotators': 3, 'scores': {'annot0': 5.0...",5
4,4,ocean of lies,Once upon a time there was an ocean of lies. S...,{'sentences': ['Once upon a time there was an ...,"{'num_annotators': 2, 'scores': {'annot0': 5.0...",5
...,...,...,...,...,...,...
478,478,The Amazing Secrets of Human Physiology,The human body is an amazing thing. It is able...,{'sentences': ['The human body is an amazing t...,"{'num_annotators': 3, 'scores': {'annot0': 2.0...",3
479,479,breaking a vase,"Once upon a time, a young woman broke her vase...","{'sentences': ['Once upon a time, a young woma...","{'num_annotators': 2, 'scores': {'annot0': 4.0...",4
480,480,Diamonds and rubies,"Once upon a time, two beautiful diamonds and a...","{'sentences': ['Once upon a time, two beautifu...","{'num_annotators': 5, 'scores': {'annot0': 4.0...",3
481,481,The Search for the Lost City,Lora and her team had been searching for the l...,{'sentences': ['Lora and her team had been sea...,"{'num_annotators': 3, 'scores': {'annot0': 5.0...",5


In [177]:
cohesentia_transcripts_restructured = cohesentia_transcripts[['Text','Coherence']]
cohesentia_transcripts_restructured = transcript_truncation(cohesentia_transcripts_restructured)
display(cohesentia_transcripts_restructured)

,Text,Coherence
0,David always knew he had a lot of weight to lo...,2
1,I was curious about the world and I wanted to ...,2
2,"Once upon a time, there were seven deadly sins...",2
3,Sally was a very special little girl. She had ...,5
4,Once upon a time there was an ocean of lies. S...,5
...,...,...
478,The human body is an amazing thing. It is able...,3
479,"Once upon a time, a young woman broke her vase...",4
480,"Once upon a time, two beautiful diamonds and a...",3
481,Lora and her team had been searching for the l...,5


In [178]:
transcripts = pd.concat([hanna_transcripts_restructured,cohesentia_transcripts_restructured])
transcripts.reset_index(drop = True, inplace = True)
transcripts['Coherence'] = transcripts['Coherence'].astype(int)
display(transcripts)

,Text,Coherence
0,"3,000 years have I been fighting. Every mornin...",3
1,"“Dad, you 're on TV again !” I heard Eric 's v...",4
2,"When Tyler entered the ward, his daughter Vale...",4
3,His body was failing. He had taken care of it ...,4
4,"I saw the button. It was simple, red, no words...",5
...,...,...
1534,The human body is an amazing thing. It is able...,3
1535,"Once upon a time, a young woman broke her vase...",4
1536,"Once upon a time, two beautiful diamonds and a...",3
1537,Lora and her team had been searching for the l...,5


In [179]:
transcripts.to_csv('processed_data/transcripts.csv')